In [1]:
from src.app import App
from src.obj.omdb_response import DataBaseResponse, get_by_title

In [2]:
app = App()

In [3]:
ents = App.load_entries()

## Comparing my mean rating with the IMDB rating

In [ ]:
import plotly.graph_objects as go
from tqdm import tqdm

In [ ]:
db_results = {}

for group in tqdm(app.get_groups()):
    db_res = get_by_title(group.title)
    db_results[group.title] = (group, db_res)

In [ ]:
my_ratings = []
imdb_ratings = []
titles = []


for title, res in db_results.items():
    group, db_res = res
    if db_res is None:
        continue
    if db_res.imdb_rating is None:
        print(f"IMDB rating is None for {title}")
        continue
    imdb_ratings.append(db_res.imdb_rating)
    my_ratings.append(group.mean_rating)
    titles.append(title)

In [41]:
def best_fit(x_coors, y_coors):
    assert (n := len(x_coors)) == len(y_coors)
    xbar = sum(x_coors) / len(x_coors)
    ybar = sum(y_coors) / len(y_coors)

    numer = sum([xi * yi for xi, yi in zip(x_coors, y_coors)]) - n * xbar * ybar
    denum = sum([xi**2 for xi in x_coors]) - n * xbar**2

    b = numer / denum
    a = ybar - b * xbar

    return a, b

In [42]:
a, b = best_fit(imdb_ratings, my_ratings)
x_vals = [min(imdb_ratings) - 1, max(imdb_ratings) + 1]
y_vals = [a + b * x for x in x_vals]

In [44]:
fig = go.Figure(
    data=go.Scatter(y=my_ratings, x=imdb_ratings, mode="markers", text=titles)
)
fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=y_vals,
        mode="lines",
        name=f"best fit line: y = {a:.2f} + {b:.2f}x",
    )
)
fig.update_layout(
    title="my ratings vs IMDB ratings",
    xaxis_title="IMDB rating",
    yaxis_title="My rating",
    margin=dict(l=10, r=10, t=30, b=10),
    template="plotly_dark",
    legend=dict(x=0, y=1, traceorder="normal", orientation="v"),
)
fig.show()